# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Rohr  ->  E. Rohr  |  ['E. Rohr']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
M. Fouesneau  ->  M. Fouesneau  |  ['M. Fouesneau']


J. Liu  ->  J. Liu  |  ['J. Liu']
K. Lee  ->  K. Lee  |  ['K. Lee']
K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
J. Liu  ->  J. Liu  |  ['J. Liu']
K. Lee  ->  K. Lee  |  ['K. Lee']
A. Winter  ->  A. Winter  |  ['A. Winter']
J. Li  ->  J. Li  |  ['J. Li']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']
Arxiv has 96 new papers today
          12 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/12 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2410.19900


extracting tarball to tmp_2410.19900...

 done.


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure figures/TNG-Cluster_CoolGasDensity-M200cz01e15_evolution.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\columnwidth]{figures/TNG-Cluster_CoolGasDensity-M200cz01e15_evolution.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure figures/TNG-Cluster_M200ct_ICMCGMt_fICMCGMt_CosmicTime-Redshift_Evolution.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegra

...

/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 4 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2410.21147/main.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


J. Neumann  ->  J. Neumann  |  ['J. Neumann']


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure c1_vs_vt
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure c1_vs_vt as tmp_2410.21147/./c1_vs_vt.png
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure sample
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure sample as tmp_2410.21147/./sample.png
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


Found 53 bibliographic references in tmp_2410.21147/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.19900-b31b1b.svg)](https://arxiv.org/abs/2410.19900) | **The cooler past of the intracluster medium in TNG-Cluster**  |
|| <mark>E. Rohr</mark>, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2024-10-29*|
|*Comments*| *Resubmitted to MNRAS after initial positive referee report. See further TNG-Cluster papers from Marine Prunier and Urmila Chadayammuri over the next days and on the TNG-Cluster website: this https URL*|
|**Abstract**|            The intracluster medium (ICM) today is comprised largely of hot gas with clouds of cooler gas of unknown origin and lifespan. We analyze the evolution of cool gas (temperatures $\lesssim10^{4.5}$ K) in the ICM of 352 galaxy clusters from the TNG-Cluster simulations, with present-day mass $\sim10^{14.3-15.4}\,{\rm M_\odot}$. We follow the main progenitors of these clusters over the past $\sim13$ billion years (since $z\lesssim7$) and find that, according to TNG-Cluster, the cool ICM mass increases with redshift at fixed cluster mass, implying that this cooler past of the ICM is due to more than just halo growth. The cool cluster gas at $z\lesssim0.5$ is mostly located in and around satellite galaxies, while at $z\gtrsim2$ cool gas can also accrete via filaments from the intergalactic medium. Lower-mass and higher-redshift clusters are more susceptible to cooling. The cool ICM mass correlates with the number of gaseous satellites and inversely with the central supermassive black hole (SMBH) mass. The average number of gaseous satellites decreases since $z=2$, correlating with the decline in the cool ICM mass over cosmic time, suggesting a link between the two. Concurrently, kinetic SMBH feedback shifts the ICM temperature distribution, decreasing the cool ICM mass inside-out. At $z\approx0.5$, the predicted MgII column densities are in the ballpark of recent observations, where satellites and other halos contribute significantly to the total MgII column density. Suggestively, a non-negligible amount of the ICM cool gas forms stars in-situ at early times, reaching $\sim10^{2}\,{\rm M_\odot}\,{\rm yr^{-1}}$ and an H$\alpha$ surface brightness of $\sim10^{-17}\,{\rm~erg\,s^{-1}\,cm^{-2}\,arcsec^{-2}}$ at $z\approx2$, detectable with Euclid and JWST.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.21147-b31b1b.svg)](https://arxiv.org/abs/2410.21147) | **On the role of non-circular motions in MaNGA galaxies I: global properties**  |
|| C. Lopez-Coba, L. Lin, <mark>J. Neumann</mark>, M. A. Bershady |
|*Appeared on*| *2024-10-29*|
|*Comments*| *13 pages, 10 figures. Accepted for publication in ApJ*|
|**Abstract**|            Non-circular (NC) motions represent the imprints of non-axisymmetric structures in galaxies, providing opportunities to study the physical properties of gas departing from circular rotation. In this work, we have conducted a systematic study of the non-circular motions in a sample of 1624 gas-rich disk galaxies from the MaNGA MPL-11. By using the H$\alpha$ velocity as a tracer of the disk rotation, we find indications that the amplitude of the non-circular motions is related to the stellar mass, with the low mass and late-type galaxies the most affected. In our sample, we find ratios of circular to non-circular rotation ranging from 5% to 20%. By implementing harmonic models to include NC motions associated with spiral arms and stellar bars, we find that the rotational curves traced with H$\alpha$ are barely affected by the NC induced by these structures. Consequently, in our sample, we do not find evidence that NC motions contribute to the scatter of the stellar Tully-Fisher relation. Our results suggest that non-circular motions might have a more localized effect in galaxies rather than a global one.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.20819-b31b1b.svg)](https://arxiv.org/abs/2410.20819) | **Analyzing the Abundance Discrepancy Problem in HII Regions with Photoionization Modeling**  |
|| A. Nemer, et al. -- incl., <mark>K. Kreckel</mark> |
|*Appeared on*| *2024-10-29*|
|*Comments*| **|
|**Abstract**|            Understanding the complex ionization structure and chemical composition of \hii\ regions poses a significant challenge in astrophysics. The abundance discrepancy problem, characterized by inconsistencies between abundances derived from recombination lines (RLs) and collisionally excited lines (CELs), has long been a puzzle in the field. In this theoretical study, we present novel photoionization models that incorporate temperature, density, and chemical inhomogeneities within a single cloud to comprehensively address this discrepancy. By accounting for the intricate interplay between ionization, excitation, and chemistry, our models successfully reproduce both observed RLs and CELs with with an average difference between our models and the observations of 25% -- within uncertainties inherent in Galactic archival long-slit and new SDSS-V Local Volume Mapper observations. Through comparisons between generic inhomogeneous model predictions and observations, demonstrating the ability of our theoretical framework to analyze the abundance discrepancy problem within \hii\ regions. Our results highlight the importance of incorporating spatially resolved temperature, density, and chemical structures when interpreting the physical processes governing emission line spectra in these astrophysical environments.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.20834-b31b1b.svg)](https://arxiv.org/abs/2410.20834) | **KIC 10855535: An elegant Delta Scuti pulsator with Amplitude and Phase Modulation**  |
|| L. Shen, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-10-29*|
|*Comments*| **|
|**Abstract**|            We investigated the pulsating behavior of KIC 10855535 using Kepler 4-year long cadence data. Two independent frequencies were detected: a pulsation frequency F0 = 17.733260(5)d-1 and a low frequency f8=0.412643(8)d-1 We identify F0 as the fundamental frequency, at which a equidistant quintuplet is centered, suggesting that the star orbits in a binary system. The fitted orbital parameters align well with those reported in previous literature. Long-term phase modulation caused by binarity has been confirmed by considering TESS light curve. Through adjusting light time via removing the light time effect, we measured a linear change in period of order $\dot{P}/P \simeq 1.44\times 10^{-7}yr^{-1}$, a value that could be indicative of stellar evolution. The star also exhibits a gradual and stable amplitude growth, thereby raising the possibility of structural changes during its evolution. We attributed f8 and its two harmonics to rotation and surface spots, with further analysis suggesting evolving characteristics over time. Based on the hypothesis, KIC 10855535 may rotate slowly for its type, with a speed of 37(2)km/s. Overall, KIC 10855535 presents an exceptionally clean spectrum and a relatively slow rotation as a {\delta} Sct pulsator, exhibiting a single pulsation mode that undergoes both amplitude and phase modulation.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.20842-b31b1b.svg)](https://arxiv.org/abs/2410.20842) | **Implication of a galaxy-scale negative feedback by one of the most powerful multi-phase outflows in a hyper-luminous infrared galaxy at the intermediate redshift**  |
|| X. Chen, et al. -- incl., <mark>K. Lee</mark> |
|*Appeared on*| *2024-10-29*|
|*Comments*| *35 pages, 26 figures, submitted to ApJ*|
|**Abstract**|            Powerful, galactic outflows driven by Active Galactic Nuclei (AGNs) are commonly considered as a main mechanism to regulate star formation in massive galaxies. Ultra- and hyper-luminous IR galaxies (U/HyLIRGs) are thought to represent a transition phase of galaxies from a rapidly growing period to a quiescent status as gas swept out by outflows, providing a laboratory to investigate outflows and their feedback effects on the hosts. In this paper we report recent Gemini and ALMA observations of a HyLIRG, J1126 at $z=0.46842$, which has been identified with a puzzling co-existence of a fast ionized outflow ($>2000$ km s$^{-1}$) and an intense starburst (star formation rate of 800 $M_{\odot}$ yr$^{-1}$). The Gemini observation shows the fast ionized outflow is extended to several kpc with a mass-loss rate of 180 $M_{\odot}$ yr$^{-1}$. A massive molecular outflow with a high mass-loss rate (2500 $M_{\odot}$ yr$^{-1}$) is revealed by ALMA. The multi-phase outflows show large factors of momentum boost and loading of kinetic power, indicating a driving by thermal pressure of a nuclear hot wind and/or radiation pressure of a highly obscured AGN. In addition to ejection of kinetic energy, it is also found that the powerful outflow can induce an ionizing shock in the galaxy disk and enhance the excitation and dissociation of molecular gas. The powerful outflow probably results in an instantaneous negative feedback and shows potential to regulate the host growth in a long term.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.21018-b31b1b.svg)](https://arxiv.org/abs/2410.21018) | **A tell-tale tracer for externally irradiated protoplanetary disks: comparing the [CI] 8727 A line and ALMA observations in proplyds**  |
|| M.-L. Aru, et al. -- incl., <mark>A. Winter</mark> |
|*Appeared on*| *2024-10-29*|
|*Comments*| *Accepted to A&A on October 25, 2024*|
|**Abstract**|            The evolution of protoplanetary disks in regions with massive OB stars is influenced by externally driven winds that deplete the outer parts of disks. These winds have previously been studied via forbidden oxygen emission lines, which also arise in isolated disks in low-mass star forming-regions (SFRs) with weak external UV fields in photoevaporative or magnetic (internal) disk winds. It is crucial to determine how to disentangle external winds from internal ones. Here, we report a proxy for unambiguously identifying externally driven winds with a forbidden line of neutral atomic carbon, [C i] 8727 A. We compare for the first time the spatial location of the emission in the [O i] 5577 A, [O i] 6300 A, and [C i] 8727 A lines traced by VLT/MUSE-NFM, with the ALMA Band 7 continuum disk emission in a sample of 12 proplyds in the Orion Nebula Cluster (ONC). We confirm that the [O i] 5577 A emission is co-spatial with the disk emission, whereas the [O i] 6300 A is emitted both on the disk surface and on the ionization front of the proplyds. We show for the first time that the [C i] 8727 A line is also co-spatial with the disk surface in proplyds, as seen in the MUSE and ALMA data comparison. To verify whether the [C i] 8727 A line is detected in regions where external photoevaporation is not expected, we examine VLT/X-Shooter spectra for young stars in low-mass SFRs. Although the [O i] lines are well detected in all these targets, there is <<10% detection rate in the case of the [C i] 8727 A line. This number increases substantially to a ~40% detection rate in sigma-Orionis, a region with intermediate UV radiation. The spatial location of the [C i] 8727 A line emission and the lack of its detection in isolated disks in low-mass SFRs strongly suggest that this line is a tell-trace tracer of externally driven photoevaporative winds, which agrees with recent excitation models.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.21038-b31b1b.svg)](https://arxiv.org/abs/2410.21038) | **Analysis of the possible detection of the pulsar wind nebulae of PSR J1208-6238, J1341-6220, J1838-0537 and J1844-0346**  |
|| W. Zhang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-10-29*|
|*Comments*| *9 pages, 6 figures*|
|**Abstract**|            Context. Pulsar wind nebulae (PWNe) are a source of very high energy (VHE) radiation that can reach up to TeV and even PeV energies. Our work uses the pulsar tree, a graph theory tool recently presented to analyze the pulsar population to select the candidates of interest. Aims. We aim to discover possible detectable PWNe. We also aim to test to what extent the pulsar tree groups detectable PWNe despite it considering only pulsars' intrinsic properties. Methods. We select four pulsars as candidates for TeV PWNe based on their positions in the pulsar tree. Using observed and assumed ranges of values for relevant parameters, we anticipate possible spectral energy distributions (SEDs) of the PWNe of four pulsars (PSR J1208-6238, J1341-6220, J1838-0537, and J1844-0346) via a detailed time-dependent, leptonic model that was already found appropriate to describe almost all other detected nebulae. Results. We estimate the likelihood of detection for the four candidates studied by comparing the TeV fluxes predicted by the possible models with the sensitivities of different observatories. In doing so, we provide context for analyzing the advantages and caveats of the pulsar tree position as a marker for properties that go beyond the intrinsic features of pulsars that are considered in producing it.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.20454-b31b1b.svg)](https://arxiv.org/abs/2410.20454) | **The Local Ultraviolet to Infrared Treasury I. Survey Overview of the Broadband Imaging**  |
|| K. M. Gilbert, et al. -- incl., <mark>M. Fouesneau</mark> |
|*Appeared on*| *2024-10-29*|
|*Comments*| *48 pages, 14 figures, 8 tables, accepted for publication in ApJS*|
|**Abstract**|            The Local Ultraviolet to Infrared Treasury (LUVIT) is a Hubble Space Telescope program that combines newly acquired data in the near ultraviolet (NUV), optical, and near infrared (NIR) with archival optical and NIR imaging to produce multiband panchromatic resolved stellar catalogs for 23 pointings in 22 low-mass, star-forming galaxies ranging in distance from the outskirts of the Local Group to ~3.8 Mpc. We describe the survey design, detail the LUVIT broadband filter observations and the archival datasets included in the LUVIT reductions, and summarize the simultaneous multiband data reduction steps. The spatial distributions and color-magnitude diagrams (CMDs) from the resulting stellar catalogs are presented for each target, from the NUV to the NIR. We demonstrate in which regions of the CMDs stars with NUV and optical, optical and NIR, and NUV through NIR detections reside. For each target, we use the results from artificial star tests to measure representative completeness, bias, and total photometric uncertainty as a function of magnitude in each broadband filter. We also assess which LUVIT targets have significant spatial variation in the fraction of stars recovered at a given magnitude. The panchromatic LUVIT stellar catalogs will provide a rich legacy dataset for a host of resolved stellar population studies.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2410.19900/figures/TNG-Cluster_M200cz01e15_ICMGasTemperatureHistogramMass_evolution.png', '', '', '', '', '']
copying  tmp_2410.19900/figures/TNG-Cluster_M200cz01e15_ICMGasTemperatureHistogramMass_evolution.png to _build/html/
file not found 
file not found 
file not found 
file not found 
file not found 
exported in  _build/html/2410.19900.md
    + _build/html/tmp_2410.19900/figures/TNG-Cluster_M200cz01e15_ICMGasTemperatureHistogramMass_evolution.png
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/
found figures ['tmp_2410.21147/./kin_circular_model_manga-8147-12703.png', 'tmp_2410.21147/./kin_circular_model_manga-10842-12704.png', 'tmp_2410.21147/./violin_manga.png', 'tmp_2410.21147/./kin_hrm_manga-8147-12703.png', 'tmp_2410.21147/./kin_hrm_manga-10842-12704.png', 'tmp_2410.21147/./q95Mstar.png', 'tmp_2410.21147/./sfr.png', 'tmp_2410.21147/./ssfr.png']
copying  tmp_2410.21147/./kin_circular_model_manga-8147-12703.png t

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]{\href{https://orcid.org/#1}{\includegraphics[scale=0.08]{ORCID-iD_icon-128x128.png}}}$
$\newcommand$
$\newcommand{\mvir}{M_{\rm 200c}}$
$\newcommand{\mvirhost}{\mvir^{\rm host}}$
$\newcommand{\rvir}{R_{\rm 200c}}$
$\newcommand{\rvirhost}{\rvir^{\rm host}}$
$\newcommand{\mgas}{M_{\rm gas}}$
$\newcommand{\mgassat}{M_{\rm gas}^{\rm sat}}$
$\newcommand{\mcgas}{M_{\rm CoolGas}}$
$\newcommand{\mcgassat}{\mcgas^{\rm sat}}$
$\newcommand{\mcgassub}{\mcgas^{\rm subhalo}}$
$\newcommand{\mcgasicm}{\mcgas^{\rm ICM}}$
$\newcommand{\mcgascgm}{\mcgas^{\rm CGM}}$
$\newcommand{\fcgasicm}{f_{\rm CoolGas}^{\rm ICM}}$
$\newcommand{\mstar}{M_\star}$
$\newcommand{\mstarsat}{\mstar^{\rm sat}}$
$\newcommand{\rhalfstar}{R_{\rm half,\star}}$
$\newcommand{\rgal}{R_{\rm gal}}$
$\newcommand{\dsathost}{d_{\rm sat}^{\rm host}}$
$\newcommand{\sublink}{\textsc{sublink} }$
$\newcommand{\sublinkgal}{\sublink\_\textsc{gal} }$
$\newcommand{\subfind}{\textsc{subfind} }$
$\newcommand{\tcoolgas}{T_{\rm CoolGas}}$
$\newcommand{\tvir}{T_{\rm vir}}$
$\newcommand{\tcool}{t_{\rm cool}}$
$\newcommand{\tff}{t_{\rm ff}}$
$\newcommand{\msun}{{\rm M}_\odot}$
$\newcommand{\kpc}{{\rm kpc}}$
$\newcommand$
$\newcommand$
$\newcommand{\ez}[1]{\textcolor[rgb]{1.0, 0.55, 0}{(#1)}}$
$\newcommand{\reza}[1]{\color{orange}#1\color{black}}$
$\newcommand{\rezac}[1]{\color{orange}\textit{[#1]}\color{black}}$
$\newcommand{\rezas}[1]{\color{orange}\sout{#1} \color{black}}$</div>



<div id="title">

# The cooler past of the intracluster medium in TNG-Cluster

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.19900-b31b1b.svg)](https://arxiv.org/abs/2410.19900)<mark>Appeared on: 2024-10-29</mark> -  _Resubmitted to MNRAS after initial positive referee report. See further TNG-Cluster papers from Marine Prunier and Urmila Chadayammuri over the next days and on the TNG-Cluster website: this https URL_

</div>
<div id="authors">

<mark>E. Rohr</mark>, et al. -- incl., <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** The intracluster medium (ICM) today is comprised largely of hot gas with clouds of cooler gas of unknown origin and lifespan. We analyze the evolution of cool gas (temperatures $\lesssim10^{4.5}$ K) in the ICM of 352 galaxy clusters from the TNG-Cluster simulations, with present-day mass $\sim10^{14.3-15.4} \msun$ . We follow the main progenitors of these clusters over the past $\sim13$ billion years (since $z\lesssim7$ ) and find that, according to TNG-Cluster, the cool ICM mass increases with redshift at fixed cluster mass, implying that this cooler past of the ICM is due to more than just halo growth. The cool cluster gas at $z\lesssim0.5$ is mostly located in and around satellite galaxies, while at $z\gtrsim2$ cool gas can also accrete via filaments from the intergalactic medium. Lower-mass and higher-redshift clusters are more susceptible to cooling. The cool ICM mass correlates with the number of gaseous satellites and inversely with the central supermassive black hole (SMBH) mass. $\er{The average number of gaseous satellites decreases since $z=2$, correlating with the decline in the cool ICM mass over cosmic time, suggesting a link between the two.}$ Concurrently, kinetic SMBH feedback shifts the ICM temperature distribution, decreasing the cool ICM mass inside-out. At $z\approx0.5$ , the predicted $\ion{Mg}{ii}$ column densities are in the ballpark of recent observations, where satellites and other halos contribute significantly to the total $\ion{Mg}{ii}$ column density. Suggestively, a non-negligible amount of the ICM cool gas forms stars in-situ at early times, reaching $\sim10^{2} \msun{\rm yr^{-1}}$ and an H $\alpha$ surface brightness of $\sim10^{-17} {\rm erg s^{-1} cm^{-2} arcsec^{-2}}$ at $z\approx2$ , detectable with Euclid and JWST.

</div>

<div id="div_fig1">

<img src="tmp_2410.19900/figures/TNG-Cluster_M200cz01e15_ICMGasTemperatureHistogramMass_evolution.png" alt="Fig6.1" width="50%"/><img src="" alt="Fig6.2" width="50%"/>

**Figure 6. -** ** The evolution of the ICM temperatures, cool gas masses, cool ICM to total halo, and cool ICM to total ICM mass fractions in TNG-Cluster since $\mathbf{z=4**$.}
    The ICM is all FoF gas in the aperture $[0.15, 1.0]$\rvir$$, excising satellites. Cool gas has temperatures $\leq 10^{4.5}$ K.
    _Main Panel_: The distribution of ICM temperatures of each cluster of $z=0$ mass $\sim10^{15} $\msun$$(51 clusters) as thin curves and the median of this sample as thick curves, where the color denotes the redshift. In the inset we show how the ICM temperature distribution varies with cluster mass today. We label prominent features in the ICM temperature distribution (see text for details; we offset the star-forming gas temperatures slightly for visibility).
    _Bottom Panels_: \er{The cool ICM mass (left), cool ICM to total cluster (center), and cool ICM to total ICM (right) mass fractions as functions of cluster mass and redshift (color) for all 352 clusters (and cluster-progenitors) in TNG-Cluster}. We plot each cluster as circles and the median trend with mass as thick curves, colored by redshift.
    According to TNG-Cluster, the ICM of cluster progenitors were cooler, having more total cool gas, cooler average temperatures, and a larger ICM mass fraction in cool gas.
     (*fig:ICMGasEvolution*)

</div>
<div id="div_fig2">

<img src="" alt="Fig10.1" width="50%"/><img src="" alt="Fig10.2" width="50%"/>

**Figure 10. -** ** According to TNG-Cluster, at a fixed cluster mass and redshift, the cool ICM mass correlates with the number of gaseous satellites, and clusters tend to have fewer gaseous satellites today than in the past.**_Top Panels_: At $z=0$ and 2 (black-outlined circles, orange-outlined squares), we demonstrate that at a fixed redshift the cool ICM mass increases with the relative number of satellites, that is, with the percentage difference between the number of satellites in a given cluster and the average number in a corresponding narrow bin of halo mass. Here, we only consider satellites with a stellar mass $>10^9  $\msun$$, a gas to stellar mass fraction $>1$ per cent, and within a cluster-centric distance $<$\rvir$$.
    _Bottom Panel_: We show the evolution of the number of gaseous satellites per cluster over the past $\approx13$ billion years for all 352 clusters (thin curves), colored by their $z=0$ cluster mass (medians within a $z=0$ cluster mass bin as thick curves). Namely for all considered masses today, clusters have fewer gaseous satellites today than in the past at $z\approx1-2$, at least partially explaining why the cool ICM mass decreases with time.
     (*fig:ICMCGM_Nsatellites*)

</div>
<div id="div_fig3">

<img src="" alt="Fig13.1" width="50%"/><img src="" alt="Fig13.2" width="50%"/>

**Figure 13. -** **Clusters and their progenitors contain $\ion${Mg**{ii} gas in TNG-Cluster.}_Top panels_: We show the evolution of the $\ion${Mg}{ii} column density for an example cluster of $z=0$ mass $\sim 10^{15.3}  $\msun$$, including gas (top: excluding satellites; bottom: including satellites and foreground+background halos) within a line-of-sight velocity $<2000  {\rm km  s^{-1}}$ of the BCG. The annotations and method are as in Fig. \ref{fig:CoolGasSurfaceDensity_mosaic}.
    _Bottom panels_: The $\ion${Mg}{ii} column density radial profiles in clusters of present-day mass $\sim10^{15}  $\msun$$ across cosmic time (left) and for all clusters of mass $\sim10^{13.75-15.25}  $\msun$$ at $z=0.5$. The thin curves show the profiles for individual clusters, including gas within the a line of sight velocity $<2000  {\rm km  s^{-1}}$ of the BCG and excluding satellites, where we include the medians as thick curves. The dashed curves show the median profiles when also including satellites. We include stacked observational comparisons of clusters from \citet[][circle]{Fresco2024}, \citet[][triangles]{Anand2022}, and \citet[][squares]{Mishra2022}, which are colored to match the average redshift (left) and cluster mass (right); see text for details.
     (*fig:mgii_comparison*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.19900"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\XS}{{\tt XookSuut}}$
$\newcommand{\xs}{{\tt XS}}$
$\newcommand{\diskfit}{{\tt DiskFit}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\PAdisk}{\phi_\mathrm{disk}^\prime}$
$\newcommand{\PAbar}{\phi_\mathrm{bar}}$
$\newcommand{\PAbarkin}{\phi_\mathrm{bar,kin}^\prime}$
$\newcommand{\PAbarphot}{\phi_\mathrm{bar,phot}^\prime}$
$\newcommand{\sn}{S/N}$
$\newcommand{\hi}{\ion{H}{1}}$
$\newcommand{\hii}{\ion{H}{2}}$
$\newcommand{\hei}{\ion{He}{i}}$
$\newcommand{\heii}{\ion{He}{ii}}$
$\newcommand{\Ni}{[\ion{N}{i}]}$
$\newcommand{\nii}{[\ion{N}{2}]}$
$\newcommand{\oi}{[\ion{O}{i}]}$
$\newcommand{\OI}{\ion{O}{i}}$
$\newcommand{\oii}{[\ion{O}{ii}]}$
$\newcommand{\nai}{\ion{Na}{i}}$
$\newcommand{\oiii}{[\ion{O}{3}]}$
$\newcommand{\sii}{[\ion{S}{ii}]}$
$\newcommand{\siii}{[\ion{S}{iii}]}$
$\newcommand{\ha}{H\alpha}$
$\newcommand{\ewha}{\mathrm{W_{H\alpha}}}$
$\newcommand{\hb}{H\beta}$
$\newcommand{\feii}{[\ion{Fe}{ii}]}$
$\newcommand{\cliii}{[\ion{Cl}{iii}]}$
$\newcommand{\ariii}{[\ion{Ar}{iii}]}$
$\newcommand{\SiII}{\ion{Si}{ii}}$
$\newcommand{\comm}[1]$</div>



<div id="title">

# On the role of non-circular motions in MaNGA galaxies I: global properties

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.21147-b31b1b.svg)](https://arxiv.org/abs/2410.21147)<mark>Appeared on: 2024-10-29</mark> -  _13 pages, 10 figures. Accepted for publication in ApJ_

</div>
<div id="authors">

C. López-Cobá, L. Lin, <mark>J. Neumann</mark>, M. A. Bershady

</div>
<div id="abstract">

**Abstract:** Non-circular (NC) motions represent the imprints of non-axisymmetric structures in galaxies, providing opportunities to study the physical properties of gas departing from circular rotation. In this work we have conducted a systematic study of the non-circular motions in a sample of 1624 gas-rich disk galaxies from the MaNGA MPL-11. By using the $\ha$ velocity as a tracer of the disk rotation, we find indications that the amplitude of the non-circular motions is related to the stellar mass, with the low mass and late-type galaxies the most affected. In our sample, we find ratios of circular to non-circular rotation ranging from 5 \% to  20 \% .By implementing harmonic models to include NC motions associated with spiral arms and stellar bars, we find that the rotational curves traced with $\ha$ are barely affected by the NC induced by these structures. Consequently, in our sample, we do not find evidence that NC motions contribute to the scatter of the stellar Tully-Fisher relation. Our results suggest that non-circular motions might have a more localized effect in galaxies rather than a global one.

</div>

<div id="div_fig1">

<img src="tmp_2410.21147/./kin_circular_model_manga-8147-12703.png" alt="Fig8.1" width="33%"/><img src="tmp_2410.21147/./kin_circular_model_manga-10842-12704.png" alt="Fig8.2" width="33%"/><img src="tmp_2410.21147/./violin_manga.png" alt="Fig8.3" width="33%"/>

**Figure 8. -** {  \it Top figures:}  Modeling of the $\ha$ velocity field adopting circular rotation only for the manga object 8147-12703 (tagged with the letter $a$), and for  10842-12704  (tagged with $b$). _ From left to right:_$gri$ colour composite image with filters extracted from the datacube; $\ha$ velocity map; best circular rotation model from $\xs$; residual map obtained from subtracting the best kinematic model to the $\ha$ velocity; rotational curve. {  \it Bottom figures:}
Half-violin plot and box plot of the residual velocities of objects $a$(blue) and $b$(red); bottom plots show the corresponding cumulative distribution function of residuals, with vertical lines showing the 10th, 50th, and 90th percentiles.
 (*fig:circular*)

</div>
<div id="div_fig2">

<img src="tmp_2410.21147/./kin_hrm_manga-8147-12703.png" alt="Fig9.1" width="50%"/><img src="tmp_2410.21147/./kin_hrm_manga-10842-12704.png" alt="Fig9.2" width="50%"/>

**Figure 9. -**  Similar as Figure \ref{fig:circular}, but for the harmonic decomposition model. Note that  $s_2(r) = c_2(r) = 0 \forall r$ as required by the model from equation. \ref{Eq:m=3}. (*fig:harmonic*)

</div>
<div id="div_fig3">

<img src="tmp_2410.21147/./q95Mstar.png" alt="Fig5.1" width="33%"/><img src="tmp_2410.21147/./sfr.png" alt="Fig5.2" width="33%"/><img src="tmp_2410.21147/./ssfr.png" alt="Fig5.3" width="33%"/>

**Figure 5. -** _ Top panel:_ 2D histogram showing the distribution of the $\eta$ parameter versus the stellar mass for the objects in the kinematic sample. Colors represent the density of points within each bin, with darker colors representing regions with a larger density of objects. The inner and outer contours enclose 68\% and 95\% percent of the data, respectively.
The black straight line represents the best Bayesian linear fit to these points, with $1\sigma$ scatter shown with gray color; the half violin plot shows the residuals respect the best fit line. _ Middle and bottom panels:_ Similar figures as above but for the integrated $\ha$ based SFR and specific SFR, respectively. (*fig:q95Mstar*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.21147"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

231  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
